In [1]:
import sqlite3
import pandas as pd
import numpy as np
from sql_metadata import Parser
from traceback import format_exc
import time

In [2]:
# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
FIRST_TIME = True

FLAG_DEBUG = False # True # 

RSI_PERIOD, RSI_AVG, RSI_BAND_WIDTH = 100, 25, 0.6
EMA_FAST, EMA_SLOW, EMA_LONG = 15, 50, 150
EMA_FAST_SCALE = 1.4  # EMA10 band half-width factor
EMA_SLOW_SCALE = 2.0 
MA_VOL = 20
SPAN, OVERLAP = 200, 15
PANID_PRICE, PANID_VOL, PANID_RSI, PANID_SIGNAL = 0, 3, 2, 1
PANEL_RATIOS = (8, 1, 8, 1)
FIGURE_WIDTH, FIGURE_HEIGHT =  17, 13
YELLOW = '#F5D928'
LIGHT_BLACK = '#8F8E83'

In [4]:
CHART_TABLE_NAME = "quote_ta"
db_name = "SP500"
file_tickers = "wl_index-SPY_SP500.csv"
file_dict = "ticker-catalog.csv"
file_db = f"db_{db_name}.sqlite"

In [5]:
def _load_stooq_data(file_path):
    """Load historical quotes from http://stooq.com/db/h
    and parse it to Pandas dataframe
    """
    df = pd.read_csv(file_path,index_col=2,parse_dates=False)
    df.reset_index(inplace=True)
    df.drop(columns=['<TICKER>', '<PER>', '<TIME>','<OPENINT>'], axis=1, inplace=True)
    # Avoid reserved words like date, open via suffix '_' for original quotes
    df.rename(columns={'<DATE>' : "date_", '<OPEN>': "open_", '<HIGH>':"high_", 
                   '<LOW>':"low_", '<CLOSE>':"close_", '<VOL>':"volume_"} ,inplace=True)
    df.set_index("date_", inplace=True)
    return df


def _ta_RSI(df, n=14):
    """Calculate Technical Analysis indicator RSI, 0-centered
    """
    # https://github.com/wgong/mplfinance/blob/master/examples/rsi.py
    diff = df.w_p.diff().values
    gains = diff
    losses = -diff
    with np.errstate(invalid='ignore'):
        gains[(gains<0)|np.isnan(gains)] = 0.0
        losses[(losses<=0)|np.isnan(losses)] = 1e-10 # we don't want divide by zero/NaN
    m = (n-1) / n
    ni = 1 / n
    g = gains[n] = np.nanmean(gains[:n])
    l = losses[n] = np.nanmean(losses[:n])
    gains[:n] = losses[:n] = np.nan
    for i,v in enumerate(gains[n:],n):
        g = gains[i] = ni*v + m*g
    for i,v in enumerate(losses[n:],n):
        l = losses[i] = ni*v + m*l
    rs = gains / losses
    return 50 - (100/(1+rs))



def _calculate_ta(df,rsi_period=RSI_PERIOD, avg_period=RSI_AVG, band_width=RSI_BAND_WIDTH):
    """Calculate Technical Analysis indicators such as 
        - EMA (fast,slow,long), EMA-band, 
        - RSI, RSI-band,
        - Vol-avg
    """
    
    df["w_p"] = 0.25*(2*df["close_"] + df["high_"] + df["low_"])
    df["ema_fast"] = df.w_p.ewm(span=EMA_FAST).mean()
    df["ema_slow"] = df.w_p.ewm(span=EMA_SLOW).mean()
    df["ema_long"] = df.w_p.ewm(span=EMA_LONG).mean()

    # range
    hl_mean_fast = (df.high_ - df.low_).ewm(span=int(EMA_FAST/2)).mean()
    df["ema_fast_u"] =  df.ema_fast + 0.5*hl_mean_fast * EMA_FAST_SCALE
    df["ema_fast_d"] =  df.ema_fast - 0.5*hl_mean_fast * EMA_FAST_SCALE

    hl_mean_slow = (df.high_ - df.low_).ewm(span=int(EMA_SLOW/2)).mean()
    df["ema_slow_u"] =  df.ema_slow + 0.5*hl_mean_slow * EMA_SLOW_SCALE
    df["ema_slow_d"] =  df.ema_slow - 0.5*hl_mean_slow * EMA_SLOW_SCALE

    # trim volume to avoid exponential form
    df['vol'] = df['volume_'] / 1000000.0
    df["vol_avg"] = df.vol.ewm(span=MA_VOL).mean()

    df["rsi"] = _ta_RSI(df,n=rsi_period)
    df["rsi_avg"] = df.rsi.ewm(span=avg_period).mean()
    df["rsi_u"] = df["rsi_avg"] + band_width 
    df["rsi_d"] = df['rsi_avg'] - band_width 
    df["rsi_signal"] = df["rsi"] - df["rsi_avg"]
    
    df["del_ema_slow"] = df.ema_slow.diff().values
    df["del_ema_long"] = df.ema_long.diff().values
    df["del_rsi_avg"] = df.rsi_avg.diff().values
    
    return df

def _display_df(df, nrows=3):
    print(df.head(nrows))
    print("\n",3*"..","\n")
    print(df.tail(nrows))

In [6]:
create_table_sql = f'''CREATE TABLE {CHART_TABLE_NAME}
    (ticker text
    
    , date_ text
	, open_ real
	, high_ real
	, low_ real
	, close_ real
	, volume_ INTEGER
    
	, w_p real
	, ema_fast real
	, ema_slow real
	, ema_long real
	, ema_fast_u real
	, ema_fast_d real
	, ema_slow_u real
	, ema_slow_d real
	, vol real
	, vol_avg real
	, rsi real
	, rsi_avg real
	, rsi_u real
	, rsi_d real
	, rsi_signal real
	, del_ema_slow real
	, del_ema_long real
	, del_rsi_avg real 
)
'''

# https://www.tutorialspoint.com/sqlite/sqlite_indexes.htm
create_index_sql = f'''
Create unique index if not exists
ticker_dt on {CHART_TABLE_NAME} ( ticker , date_ )
'''

In [7]:
parser = Parser(create_table_sql)
table_name = parser.tables[0]
table_columns = parser.columns   # open is a key-word, skipped
print(f"table_name: {table_name} \n\ncolumns: {table_columns}")

table_name: quote_ta 

columns: ['ticker', 'date_', 'open_', 'high_', 'low_', 'close_', 'volume_', 'w_p', 'ema_fast', 'ema_slow', 'ema_long', 'ema_fast_u', 'ema_fast_d', 'ema_slow_u', 'ema_slow_d', 'vol', 'vol_avg', 'rsi', 'rsi_avg', 'rsi_u', 'rsi_d', 'rsi_signal', 'del_ema_slow', 'del_ema_long', 'del_rsi_avg']


In [8]:
numeric_cols = ['w_p', 'ema_fast', 'ema_slow', 'ema_long', 'ema_fast_u', 'ema_fast_d', 'ema_slow_u', 'ema_slow_d', 
                'vol', 'vol_avg', 'rsi', 'rsi_avg', 'rsi_u', 'rsi_d', 'rsi_signal', 
                'del_ema_slow', 'del_ema_long', 'del_rsi_avg']

#### Create table if first-time

In [9]:
if FIRST_TIME:
    conn = sqlite3.connect(file_db)
    cur = conn.cursor()

    # Create table
    cur.execute(create_table_sql)

    conn.commit()
    conn.close()

In [10]:
df_ticker = pd.read_csv(file_tickers)
df_meta = pd.read_csv(file_dict)

In [11]:
_display_df(df_ticker)

  Ticker
0    SPY
1    XLE
2    XLB

 ...... 

    Ticker
497   ZBRA
498   ZION
499    ZTS


In [12]:
tickers = df_ticker["Ticker"].to_list()

In [13]:
len(tickers), tickers[:5]

(500, ['SPY', 'XLE', 'XLB', 'XME', 'XLK'])

In [14]:
conn = sqlite3.connect(file_db)

In [15]:
delta_ts = 0
for ticker in tickers:
    ts_begin = time.time()
    
    try:
        dd = df_meta[df_meta['ticker'] == ticker].to_dict('records')[0]
    except:
        print(f"ticker: {ticker}\n{format_exc()}")
        continue
    
    # load quotes, TA, round
    try:
        df = _load_stooq_data(dd['file_path'])
    except:
        print(f"ticker: {ticker}\nfile_path: {dd['file_path']}\n{format_exc()}")
        continue
        
    df = _calculate_ta(df)
    df[numeric_cols] = np.round(df[numeric_cols], 4)
    df.reset_index(inplace=True)
    df['ticker'] = ticker
    
    # save to Sqlite
    df.to_sql(table_name, conn, if_exists='append', index=False)
    ts_end = time.time()
    print(f"ticker = {ticker} done in {(ts_end-ts_begin):.2f} sec")
    delta_ts += (ts_end-ts_begin)
    
print(f"number of tickers={len(tickers)}, calculations done in {delta_ts:.2f} sec")

ticker = SPY done in 0.26 sec
ticker = XLE done in 0.23 sec
ticker = XLB done in 0.15 sec
ticker = XME done in 0.14 sec
ticker = XLK done in 0.13 sec
ticker = XLF done in 0.16 sec
ticker = XLV done in 0.20 sec
ticker = XLI done in 0.17 sec
ticker = XLP done in 0.14 sec
ticker = XLY done in 0.14 sec
ticker = XLC done in 0.10 sec
ticker = XLU done in 0.14 sec
ticker = XLRE done in 0.08 sec
ticker = A done in 0.15 sec
ticker = AAL done in 0.12 sec
ticker = AAP done in 0.17 sec
ticker = AAPL done in 0.29 sec
ticker = ABBV done in 0.12 sec
ticker = ABC done in 0.18 sec
ticker = ABMD done in 0.12 sec
ticker = ABT done in 0.26 sec
ticker = ACN done in 0.12 sec
ticker = ADBE done in 0.22 sec
ticker = ADI done in 0.24 sec
ticker = ADM done in 0.21 sec
ticker = ADP done in 0.26 sec
ticker = ADSK done in 0.25 sec
ticker = AEE done in 0.23 sec
ticker = AEP done in 0.32 sec
ticker = AES done in 0.20 sec
ticker = AFL done in 0.23 sec
ticker = AIG done in 0.25 sec
ticker = AIV done in 0.17 sec
ticker

ticker = KMX done in 0.15 sec
ticker = KO done in 0.28 sec
ticker = KR done in 0.29 sec
ticker = KSS done in 0.17 sec
ticker = L done in 0.12 sec
ticker = LDOS done in 0.14 sec
ticker = LEG done in 0.22 sec
ticker = LEN done in 0.12 sec
ticker = LH done in 0.15 sec
ticker = LHX done in 0.12 sec
ticker = LIN done in 0.06 sec
ticker = LKQ done in 0.12 sec
ticker = LLY done in 0.24 sec
ticker = LMT done in 0.25 sec
ticker = LNC done in 0.27 sec
ticker = LNT done in 0.12 sec
ticker = LOW done in 0.21 sec
ticker = LRCX done in 0.20 sec
ticker = LUV done in 0.25 sec
ticker = LVS done in 0.14 sec
ticker = LW done in 0.07 sec
ticker = LYB done in 0.09 sec
ticker = LYV done in 0.11 sec
ticker = MA done in 0.12 sec
ticker = MAA done in 0.14 sec
ticker = MAR done in 0.22 sec
ticker = MAS done in 0.26 sec
ticker = MCD done in 0.28 sec
ticker = MCHP done in 0.17 sec
ticker = MCK done in 0.19 sec
ticker = MCO done in 0.14 sec
ticker = MDLZ done in 0.11 sec
ticker = MDT done in 0.26 sec
ticker = MET 

#### Create index after loading quotes

In [16]:
if FIRST_TIME:
    conn = sqlite3.connect(file_db)
    cur = conn.cursor()

    # Create table
    cur.execute(create_index_sql)

    conn.commit()
    conn.close()